In [ ]:
#importing libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from pandas.plotting import hist_frame
from numpy import set_printoptions
from pandas import read_csv
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import scipy
import ast
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sbn
import csv


In [ ]:
#loading csv containing song attributes with pandas
filename = 'songAttributes.csv'
data = read_csv(filename,on_bad_lines = 'skip',encoding = 'latin1')
shape = data.shape
print(shape)
types = data.dtypes
print(types)

In [ ]:
#loading the csv containing user reviews with pandas
filename = 'userRatingsDataset.csv'
userData = read_csv(filename,on_bad_lines = 'skip')
shape = userData.shape
print(shape)
types = userData.dtypes
print(types)

(101192, 4)
userID       int64
songID       int64
rating       int64
songName    object
dtype: object


In [ ]:
#loading the csv containing user ratings with pandas
filename = 'songAttributes.csv'
data = read_csv(filename,on_bad_lines = 'skip',encoding = 'latin1')
shape = data.shape
print(shape)
types = data.dtypes
print(types)

In [ ]:
set_option('display.width',100)
set_option('display.precision',3)
description = data.describe()
print(description)

In [ ]:
print('Null values:\n',data.isnull().sum(),'\nDuplicate song names:\n',data.duplicated('song_name').sum())



In [ ]:
userData.isnull().sum()
userData.duplicated('userID').sum()

91072

In [ ]:
#dropping rows with duplicate song names and null values
data=data.drop_duplicates('song_name',keep= 'first',ignore_index = True)
data.dropna(inplace = True)
print(data.shape)

(8344, 10)


In [ ]:
#dropping columns I dont need
drop_columns =['loudness','mode','tempo','key','type','id','uri','track_href','analysis_url','duration_ms','time_signature']
data = data.drop(drop_columns,axis = 1)
data = data.drop(data.columns[0],axis = 1)

In [ ]:
"""cv = CountVectorizer()
ansm = cv.fit_transform(data['artist'])
cosine_sim = cosine_similarity(ansm,ansm)
print(cosine_sim)"""

In [ ]:
df = pd.DataFrame(data)
title_array = df.loc[:,'song_name'].values
print(title_array)
artist_array = df.loc[:,'artist'].values
genre_array = df.loc[:,'genre'].values
#print(artist_array)

['Dark River - Festival Version' 'Derp - Original Mix'
 'One Kiss (with Dua Lipa)' ... 'One Of Us - Short Version'
 'Eyes Without A Face' 'Rebel Yell']


In [ ]:
genre_array.drop_duplicates()

['absolute music' 'absolute music' 'absolute music' ... 'rock' 'rock'
 'rock']


In [ ]:
def same_artist_songs(title,count):
    k=0
    rec_array = []
    req_nr = 1

    #I obtain the index of the song
    idx = data[data["song_name"] == title].index.tolist()[0]
    #I obtain the artist of the song based on the index
    artistf = data.iloc[idx].artist
    #I loop through the artists array
    for x in range (0,len(artist_array)):
        #I check if the artist in the array is the same as the artist of the song given as parameter and if the count isn't reached
        if artist_array[x] == artistf and k<count:
            #if the above condition is satisfied the song is added to the array with songs
            rec_array.append(title_array[x])
            k +=1
         #I check if the last value of the artist array was reached while the value of k didn't reach the value of count
        if x==len(artist_array)-1 and k<count:
            print(f'There are only {k} songs by {artistf}')
            req_nr = 0
            break


    if req_nr == 1:
        if count == 1:
          print(f'Here is {count} song similar to {title} by {artistf}:')
        elif count > 1:
          print(f'Here are {count} songs similar to {title} by {artistf}:')
        else:
          print('Error')

    #I print all the values of the array with songs of the same artist
    for j in range(0,len(rec_array)):
        print(f'{j+1}.{rec_array[j]}')

In [ ]:
def same_genre_songs(title,count):
    k=0
    rec_array = []
    req_nr = 1

    #I obtain the index of the song
    idx = data[data["song_name"] == title].index.tolist()[0]
    #I obtain the genre of the song based on the index
    genretf = data.iloc[idx].genre
    #I loop through the genre array
    for x in range (0,len(genre_array)):
        #I check if the genre in the array is the same as the genre of the song given as parameter and if the count isn't reached
        if genre_array[x] == genretf and k<count:
            #if the above condition is satisfied the song is added to the array with songs
            rec_array.append(title_array[x])
            k +=1
         #I check if the last value of the genre array was reached while the value of k didn't reach the value of count
        if x==len(genre_array)-1 and k<count:
            print(f'There are only {k} {genretf} songs')
            req_nr = 0
            break


    if req_nr == 1:
        if count == 1:
          print(f'Here is {count} {genretf} song similar to {title}')
        elif count > 1:
          print(f'Here are {count} {genretf} songs similar to {title}:')
        else:
          print('Error')

    #I print all the values of the array with songs of the same genre
    for j in range(0,len(rec_array)):
        print(f'{j+1}.{rec_array[j]}')

In [ ]:
#@title Recommendation of songs with the same artist

Title ="Mask Off" #@param {type:"string"}
count = 15 #@param {type:"raw"}
same_artist_songs(Title,count)


In [ ]:
print(data)

In [ ]:
def similar_songs_rec(data,song,artist,count):
  try:
      #storing the name of the song and the artist
      artist_and_song_data = data[(data['song_title']==song) &(data['artist']==artist)]
      #creating a copy of the dataset
      similar_songs = data.copy()
      #storing in a table all the rows for the specified columns
      sound_properties = similar_songs.loc[:,['acousticness','danceability','duration_ms','energy','instrumentalness','key','liveness','loudness','mode','speechiness','tempo','time_signature','valence']]

      similar_songs['Similarity with song'] = cosine_similarity(sound_properties,sound_properties.to_numpy()[ artist_and_song_data.index[0],None]).squeeze()
      #renaming the name column for the similar_songs table
      similar_songs.rename(columns={'song_title':f'Songs similar to {song}'},inplace = True)
      #sorting the values of the similar_songs table in descending order
      similar_songs = similar_songs.sort_values(by = 'Similarity with song',ascending = False)

      similar_songs = similar_songs[['artist',f'Songs similar to {song}']]



      similar_songs.reset_index(drop = True,inplace = True)
      #print(similar_songs)
      return similar_songs.iloc[1:count+1]

  except:
      print("This song is not included in the dataset")


In [ ]:
#@title Recommendation of songs with similar attributes

Song = "Mask Off" #@param {type:"string"}
Artist = "Future" #@param {type:"string"}
Number_of_songs = 6 #@param {type:"raw"}
similar_songs_rec(data,Song,Artist,Number_of_songs)

In [ ]:
#print(data)
display(data)

In [ ]:
dropped_df =pd.DataFrame(data)

In [ ]:
display(dropped_df)

In [ ]:
array = dropped_df.values

#binarization of danceability
dance_array= array[:,0:1]
#print(x)
binarizer = Binarizer(threshold= 0.5).fit(dance_array)
binary_dance_array = binarizer.transform(dance_array)
set_printoptions(precision = 3)
print('Danceability:')
print(binary_dance_array[0:20,:])
print("\n")

#binarization of energy
energy_array= array[:,1:2]
#print(x)
binarizer = Binarizer(threshold= 0.5).fit(energy_array)
binary_energy_array = binarizer.transform(energy_array)
set_printoptions(precision = 3)
print('Energy:')
print(binary_energy_array[0:20,:])
print("\n")

#binarization of speechiness
speech_array = array[:,2:3]
#print(speech_array)
binarizer = Binarizer(threshold = 0.66).fit(speech_array)
binary_speech_array = binarizer.transform(speech_array)
print('Speechiness:')
print(binary_speech_array[0:20])
print("\n")

#binarization of acousticness
acoustic_array= array[:,3:4]
#print(x)
binarizer = Binarizer(threshold= 0.5).fit(acoustic_array)
binary_acoustic_array = binarizer.transform(acoustic_array)
set_printoptions(precision = 3)
print('Acousticness:')
print(binary_acoustic_array[0:20,:])
print("\n")

#binarization of instrumentalness
instrumental_array= array[:,4:5]
#print(x)
binarizer = Binarizer(threshold= 0.5).fit(instrumental_array)
binary_instrumental_array = binarizer.transform(instrumental_array)
set_printoptions(precision = 3)
print('Instrumentalness:')
print(binary_instrumental_array[0:20,:])
print("\n")

#binarization of liveness
live_array = array[:,5:6]
#print(live_array)
binarizer = Binarizer(threshold = 0.8).fit(live_array)
binary_live_array = binarizer.transform(live_array)
print('Liveness:')
print(binary_live_array[0:20])
print("\n")

#binarization of valence
valence_array = array[:,6:7]
#print(valence_array)
binarizer = Binarizer(threshold = 0.5).fit(valence_array)
binary_valence_array = binarizer.transform(valence_array)
print('Valence:')
print(binary_valence_array[0:20])


In [ ]:
display(dropped_df)

In [ ]:

def recommend_by_keywords(keywords,count):
  i=0


  for k in range(len(binary_dance_array)) :
    check =1
    for keyword in keywords:
        if keyword == "danceable":
          if binary_dance_array[k,0:1] == 0:
            check = 0
        if keyword == "acoustic":
          if binary_acoustic_array[k,0:1] == 0:
            check = 0
        if keyword == "instrumental":
          if binary_instrumental_array[k,0:1] == 0:
            check = 0
        if keyword == "live":
          if binary_live_array[k,0:1] == 0:
            check = 0
        if keyword == "energic":
          if binary_energy_array[k,0:1] == 0:
            check = 0
        if keyword == "positive":
          if binary_valence_array[k,0:1] == 0:
            check = 0
        if keyword == "negative":
          if binary_valence_array[k,0:1] == 1:
            check = 0
    if check == 1:
          print(f'{data.loc[k,"song_name"]} by {data.loc[k,"artist"]}')
          i +=1
          if i == count:
            break



In [ ]:
#@title Recommendation of songs based on keywords
Keyword = 'acoustic' #@param ["danceable","instrumental","energic","live","acoustic","positive","negative"]
Number_of_songs = 11 #@param {type:"raw"}
recommend_by_keywords(Keyword,Number_of_songs)

In [ ]:
recommend_by_keywords(["danceable","energic"],15)

In [ ]:
print(binary_dance_array[:20])
print(len(binary_dance_array))

In [ ]:
df = ps.DataFrame(data)
x=df.loc[:,'artist'].values
print(x)



In [ ]:
userDf = pd.DataFrame(userData)

In [ ]:
def similar_user_songs(id,count){




}

In [ ]:
userDf['rating'] = userDf['rating'].astype(str)
grouped_ratings = userDf.groupby('userID')['rating'].apply(','.join).reset_index()

In [ ]:
display(grouped_ratings)

In [ ]:
userDf.duplicated('songID').sum()